In [32]:
## MLB bang detection
#add data to dataframe
import os
import numpy as np
import soundfile as sf # for opening FLAC
import librosa # for MFCC
from scipy.signal import spectrogram

audio_features = [] # array of matrices (spectrograms) -> used as features for model
labels = [] # bang(1) or no bang(0)

# add training data with no bangs
path_dir_nbang = "no_bang"  # folder with training data with no anomaly present

for filename in os.listdir(path_dir_nbang):
    file_path = os.path.join(path_dir_nbang, filename)
    audio_data, sample_rate = sf.read(file_path)

    frequencies, times, spectrogram_data = spectrogram(audio_data, sample_rate)
    audio_features.append(spectrogram_data.flatten().reshape(-1))
    # use directory as name of label
    labels.append(0)

# add training data with bangs
path_dir_bang = "bang"  # folder with training data with anomaly present

for filename in os.listdir(path_dir_bang):
    file_path = os.path.join(path_dir_bang, filename)
    audio_data, sample_rate = sf.read(file_path)

    frequencies, times, spectrogram_data = spectrogram(audio_data, sample_rate)

    audio_features.append(spectrogram_data.flatten().reshape(-1))
    # use directory as name of label
    labels.append(1)

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

audio_features = np.array(audio_features, dtype="object")
labels = np.array(labels, dtype="object")

X_train, X_test, y_train, y_test = train_test_split(audio_features, labels, test_size = .2, random_state = 727)

X_train = np.concatenate(X_train).reshape(len(X_train), -1) # ensures X_train is 2d because audio_features is 1d
y_train = np.ravel(y_train).astype(int) # ensures type and 1d y_train

random_forst_classifier = RandomForestClassifier(n_estimators = 50, random_state = 727)

random_forst_classifier.fit(X_train, y_train)

X_test = np.concatenate(X_test).reshape(len(X_test), -1)
y_test = np.ravel(y_test).astype(int)

print(random_forst_classifier.score(X_test, y_test))

0.5833333333333334
